<a href="https://colab.research.google.com/github/ggwnsghgg/book_project/blob/main/2021%20/%203%20/%2030%20/%20%EC%B1%85_%EC%95%8C%EA%B3%A0%EB%A6%AC%EC%A6%98(%EC%9C%A0%EC%82%AC%EB%8F%84%20%EA%B2%80%EC%A6%9D%20%EC%88%98%EC%A0%95%EC%A4%91).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 필요한 패키지 install

In [5]:
!pip install pymysql


# 형태소 기반 토크나이징 (Konlpy)
!python3 -m pip install konlpy
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)



#한국어 문장 분리모델
!pip install kss


!sudo apt-get install curl git
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)


#mecab 코랩서 오류 안나게!
!apt-get update
!apt-get install g++ openjdk-8-jdk 
!pip3 install konlpy JPype1-py3
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

mecab-ko is already installed
mecab-ko-dic is already installed
mecab-python is already installed
Done.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
curl is already the newest version (7.58.0-2ubuntu3.12).
git is already the newest version (1:2.17.1-1ubuntu0.8).
0 upgraded, 0 newly installed, 0 to remove and 60 not upgraded.
mecab-ko is already installed
mecab-ko-dic is already installed
mecab-python is already installed
Done.
Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://security.ubuntu.com/ubuntu bionic-securit

In [6]:
import os
import kss
import re
import pandas as pd
import numpy as np
import konlpy
from tqdm import tqdm
from konlpy.tag import Mecab
import pymysql

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from google.colab import drive
drive.mount('/content/drive')



#데이터 가져오기
db = pymysql.connect(host='220.86.191.27', port=3306, user='playdata', password='playdata123!', db='bookdata', charset='utf8')

curs = db.cursor(pymysql.cursors.DictCursor)  

query = '''
select *
    from books_data'''
df_books_data = pd.read_sql(query, db)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
removal_list =  "‘, ’, ◇, ‘, ”,  ’, ', ·, \“, ·, △, ●,  , ■, (, ), \", >>, `, /, -,∼,=,ㆍ<,>, .,?, !,【,】, …, ◆,%"

def cleansing_special(sentence):
    # 특수문자를 전처리를 하는 함수
    sentence = re.sub("[.,\'\"’‘”“!?]", "", sentence)
    sentence = re.sub("[^가-힣0-9a-zA-Z\\s]", " ", sentence)
    sentence = re.sub("\s+", " ", sentence)
    
    sentence = sentence.translate(str.maketrans(removal_list, ' '*len(removal_list)))
    sentence = sentence.strip()
    
    return sentence

def cleansing_numbers(sentence):
    # 숫자를 전처리(delexicalization) 하는 함수    
    sentence = re.sub('[0-9]+', 'NUM', sentence)
    sentence = re.sub('NUM\s+', "NUM", sentence)
    sentence = re.sub('[NUM]+', "NUM", sentence)
    
    return sentence

In [5]:
# 구글 드라이브 사용시

path = '/content/drive/MyDrive/'
df_books = pd.read_excel(os.path.join(path, 'book_preprocessing.xlsx'))

In [8]:
df_books_data

,id,title,author,pub,date,price,review,rating,story,img,genre
0,1,요리코를 위해,노리즈키 린타로,모모,2020.03,13500,43,9.2,스스로 악마이자 신이 되어 딸을 살해한 남자를 심판한 아버지.비극의 출발이자 마지막...,http://image.yes24.com/goods/89392792/L,추리/미스터리
1,2,그녀는 다 계획이 있다,히가시노 게이고,하빌리스,2021.02,14220,13,9.4,하나야 보석점 살인사건을 둘러싼 과거와 미래의 사건경쾌하고 리듬감 있는 터치의 히가...,http://image.yes24.com/goods/97561319/L,추리/미스터리
2,3,블랙 쇼맨과 이름 없는 마을의 살인,히가시노 게이고,알에이치코리아(RHK),2020.11,16200,54,9.1,"아버지가 살해당했다, 그것도 결혼 소식을 알린 직후에. 행복하다고 느끼는 순간, 비...",http://image.yes24.com/goods/95712763/L,추리/미스터리
3,4,돌이킬 수 없는 약속,야쿠마루 가쿠,북플라자,2017.02,13500,217,8.2,"제51회 에드가와란포상 수상 작가 야쿠마루 가쿠의 미스터리 추리 소설. ""내 딸을 ...",http://image.yes24.com/goods/35552641/L,추리/미스터리
4,5,루팡의 딸,요코제키 다이,북플라자,2019.09,13500,31,8.5,도둑 집안의 대부가 변사체로 발견되었다!도둑 집안의 대부 이와오에게는 하나코라는 손...,http://image.yes24.com/goods/78860364/L,추리/미스터리
...,...,...,...,...,...,...,...,...,...,...,...
79770,79771,하루만 사랑해 2,,파피루스(디앤씨미디어),2003.06,8100,0,0.0,어느 날 서원은 횡단보도에서 교통사고를 당한다. 상대는 인천상고 짱인 노란머 민하루...,http://image.yes24.com/goods/375484/L,로맨스
79771,79772,서울 여자와 대구 남자가 만날 때,정민,영상노트,2003.06,8100,0,0.0,,http://image.yes24.com/goods/376458/L,로맨스
79772,79773,kiss me...,,현무,2003.06,7200,2,5.0,,http://image.yes24.com/goods/377351/L,로맨스
79773,79774,어린 엄마 2,임은희,반디출판사,2003.07,8100,0,0.0,,http://image.yes24.com/goods/377607/L,로맨스


In [9]:
df_books_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79775 entries, 0 to 79774
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   id      79775 non-null  int64  
 1   title   79775 non-null  object 
 2   author  79775 non-null  object 
 3   pub     79775 non-null  object 
 4   date    79775 non-null  object 
 5   price   79775 non-null  int64  
 6   review  79775 non-null  int64  
 7   rating  79775 non-null  float64
 8   story   79775 non-null  object 
 9   img     79775 non-null  object 
 10  genre   79775 non-null  object 
dtypes: float64(1), int64(3), object(7)
memory usage: 6.7+ MB


In [10]:
# df_books=df_books.rename(columns = {'Unnamed: 0' : 'book_id'})
#결측값 제거
df_books_data = df_books_data.dropna()

In [14]:
df_books_data.value_counts(df_books_data['rating'].values, sort=False)

0.0     60957
2.0       109
2.5         2
3.0        47
3.5         5
        ...  
9.6       252
9.7        96
9.8       115
9.9        21
10.0     1819
Length: 64, dtype: int64

In [15]:
#레이팅, 리뷰값 모두 0점인것 제거 (다행히 리뷰와 레이팅이 모두 0점인게 공통적이다)
df_books_data =df_books_data[df_books_data.review != 0]

In [17]:
df_books=df_books_data.reset_index(drop = False, inplace = False)

In [18]:
df_books

,index,id,title,author,pub,date,price,review,rating,story,img,genre
0,0,1,요리코를 위해,노리즈키 린타로,모모,2020.03,13500,43,9.2,스스로 악마이자 신이 되어 딸을 살해한 남자를 심판한 아버지.비극의 출발이자 마지막...,http://image.yes24.com/goods/89392792/L,추리/미스터리
1,1,2,그녀는 다 계획이 있다,히가시노 게이고,하빌리스,2021.02,14220,13,9.4,하나야 보석점 살인사건을 둘러싼 과거와 미래의 사건경쾌하고 리듬감 있는 터치의 히가...,http://image.yes24.com/goods/97561319/L,추리/미스터리
2,2,3,블랙 쇼맨과 이름 없는 마을의 살인,히가시노 게이고,알에이치코리아(RHK),2020.11,16200,54,9.1,"아버지가 살해당했다, 그것도 결혼 소식을 알린 직후에. 행복하다고 느끼는 순간, 비...",http://image.yes24.com/goods/95712763/L,추리/미스터리
3,3,4,돌이킬 수 없는 약속,야쿠마루 가쿠,북플라자,2017.02,13500,217,8.2,"제51회 에드가와란포상 수상 작가 야쿠마루 가쿠의 미스터리 추리 소설. ""내 딸을 ...",http://image.yes24.com/goods/35552641/L,추리/미스터리
4,4,5,루팡의 딸,요코제키 다이,북플라자,2019.09,13500,31,8.5,도둑 집안의 대부가 변사체로 발견되었다!도둑 집안의 대부 이와오에게는 하나코라는 손...,http://image.yes24.com/goods/78860364/L,추리/미스터리
...,...,...,...,...,...,...,...,...,...,...,...,...
18813,79741,79742,나는 그놈의 전부였다 1,,청어람,2003.05,7200,3,5.7,,http://image.yes24.com/goods/362884/L,로맨스
18814,79761,79762,맥그리거의 운명,노라 노버츠,신영미디어,2003.06,7650,3,7.3,돈독한 우애를 자랑하는 맥그리거 남매들! 케인의 여동생 세레나와 다이애너의 오빠가 ...,http://image.yes24.com/goods/370757/L,로맨스
18815,79762,79763,악마의 키스,,신영미디어,2003.06,8100,2,8.5,"""스테파니 로렌스가 전하는 신스터(Cynster) 가문 이야기! ?바 신스터?일당의...",http://image.yes24.com/goods/373141/L,로맨스
18816,79769,79770,하루만 사랑해 1,,파피루스(디앤씨미디어),2003.06,8100,1,7.0,어느 날 서원은 횡단보도에서 교통사고를 당한다. 상대는 인천상고 짱인 노란머 민하루...,http://image.yes24.com/goods/375482/L,로맨스


In [16]:
df_books_data.tail()

,id,title,author,pub,date,price,review,rating,story,img,genre
79741,79742,나는 그놈의 전부였다 1,,청어람,2003.05,7200,3,5.7,,http://image.yes24.com/goods/362884/L,로맨스
79761,79762,맥그리거의 운명,노라 노버츠,신영미디어,2003.06,7650,3,7.3,돈독한 우애를 자랑하는 맥그리거 남매들! 케인의 여동생 세레나와 다이애너의 오빠가 ...,http://image.yes24.com/goods/370757/L,로맨스
79762,79763,악마의 키스,,신영미디어,2003.06,8100,2,8.5,"""스테파니 로렌스가 전하는 신스터(Cynster) 가문 이야기! ?바 신스터?일당의...",http://image.yes24.com/goods/373141/L,로맨스
79769,79770,하루만 사랑해 1,,파피루스(디앤씨미디어),2003.06,8100,1,7.0,어느 날 서원은 횡단보도에서 교통사고를 당한다. 상대는 인천상고 짱인 노란머 민하루...,http://image.yes24.com/goods/375482/L,로맨스
79772,79773,kiss me...,,현무,2003.06,7200,2,5.0,,http://image.yes24.com/goods/377351/L,로맨스


In [19]:
df_mystery=df_books[df_books['genre']=='추리/미스터리']
df_horror =df_books[df_books['genre']== '공포/스릴러']
df_fantasy = df_books[df_books['genre']=='판타지']
df_martial = df_books[df_books['genre']=='무협']
df_sf = df_books[df_books['genre']=='SF']
df_history = df_books[df_books['genre']=='역사']
df_romance = df_books[df_books['genre']=='로맨스']


print(df_mystery.shape)
print(df_horror.shape)
print(df_fantasy.shape)
print(df_martial.shape)
print(df_sf.shape)
print(df_history.shape)
print(df_romance.shape)

(3203, 12)
(2575, 12)
(3385, 12)
(665, 12)
(968, 12)
(1918, 12)
(6104, 12)


In [20]:
# 형태소 작업 형태소 작업

from konlpy.tag import Mecab

df_mystery_list = list(np.array(df_mystery['story'].tolist()))
df_horror_list = list(np.array(df_horror['story'].tolist()))
df_fantasy_list = list(np.array(df_fantasy['story'].tolist()))
df_martial_list = list(np.array(df_martial['story'].tolist()))
df_sf_list = list(np.array(df_sf['story'].tolist()))
df_history_list = list(np.array(df_history['story'].tolist()))
df_romance_list = list(np.array(df_romance['story'].tolist()))

In [21]:

mecab = Mecab()

In [22]:

#미스터리
all_sentences_mystery = []
for idx, line in enumerate(df_mystery_list):
  #print(f"--미스테리소설{idx}번--")
  preprocessed = cleansing_numbers(line)
  preprocessed = cleansing_special(preprocessed)
  #명사
  preprocessed_list = ' '.join(list(set(mecab.nouns(preprocessed))))
  #print(preprocessed_list)
  all_sentences_mystery.append(preprocessed_list)

In [23]:
#호러
all_sentences_horror = []
for idx, line in enumerate(df_horror_list):
  #print(f"--호러소설{idx}번--")
  preprocessed = cleansing_numbers(line)
  preprocessed = cleansing_special(preprocessed)
  #명사
  preprocessed_list = ' '.join(list(set(mecab.nouns(preprocessed))))
  #print(preprocessed_list)
  all_sentences_horror.append(preprocessed_list)

In [24]:
#판타지
all_sentences_fantasy = []
for idx, line in enumerate(df_fantasy_list):
  #print(f"--판타지소설{idx}번--")
  preprocessed = cleansing_numbers(line)
  preprocessed = cleansing_special(preprocessed)
  #명사
  preprocessed_list = ' '.join(list(set(mecab.nouns(preprocessed))))
  #print(preprocessed_list)
  all_sentences_fantasy.append(preprocessed_list)

In [25]:
#무협
all_sentences_martial = []
for idx, line in enumerate(df_martial_list):
  #print(f"--무협소설{idx}번--")
  preprocessed = cleansing_numbers(line)
  preprocessed = cleansing_special(preprocessed)
  #명사
  preprocessed_list = ' '.join(list(set(mecab.nouns(preprocessed))))
  #print(preprocessed_list)
  all_sentences_martial.append(preprocessed_list)

In [26]:

#sf
all_sentences_sf = []
for idx, line in enumerate(df_sf_list):
  #print(f"--sf소설{idx}번--")
  preprocessed = cleansing_numbers(line)
  preprocessed = cleansing_special(preprocessed)
  #명사
  preprocessed_list = ' '.join(list(set(mecab.nouns(preprocessed))))
  #print(preprocessed_list)
  all_sentences_sf.append(preprocessed_list)

In [27]:
#역사
all_sentences_history = []
for idx, line in enumerate(df_history_list):
  #print(f"--역사소설{idx}번--")
  preprocessed = cleansing_numbers(line)
  preprocessed = cleansing_special(preprocessed)
  #명사
  preprocessed_list = ' '.join(list(set(mecab.nouns(preprocessed))))
  #print(preprocessed_list)
  all_sentences_history.append(preprocessed_list)

In [28]:
#로맨스
all_sentences_romance = []
for idx, line in enumerate(df_romance_list):
  #print(f"--연애소설{idx}번--")
  preprocessed = cleansing_numbers(line)
  preprocessed = cleansing_special(preprocessed)
  #명사
  preprocessed_list = ' '.join(list(set(mecab.nouns(preprocessed))))
  #print(preprocessed_list)
  all_sentences_romance.append(preprocessed_list)

In [29]:
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict  #단어 개수를 세고, 빈도와 리스트 셋 항목 정리리

In [30]:

print(len(all_sentences_mystery))
print(len(all_sentences_horror))
print(len(all_sentences_fantasy))
print(len(all_sentences_martial))
print(len(all_sentences_sf))
print(len(all_sentences_history))
print(len(all_sentences_romance))

3203
2575
3385
665
968
1918
6104


In [31]:

tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(all_sentences_mystery)

word2id = defaultdict(lambda : 0)
for idx, feature in enumerate(tfidf_vectorizer.get_feature_names()):
  word2id[feature] = idx

for idx, sent in enumerate(all_sentences_mystery):
    #print(f'---미스테리 소설 {idx} 번---')
    results_mystery = [(token, tfidf_matrix[idx, word2id[token]]) for token in sent.split()]
    results_mystery.sort(key = lambda element : element[1], reverse = True)
    #print(results_mystery)
    #print("\n")

In [32]:
#tfidf_vectorizer.vocabulary_  #어떤 단어가 어떤 번호를 할당받았는지!
tfidf_matrix.shape

(3203, 11156)

In [33]:

#호러
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(all_sentences_horror)

word2id = defaultdict(lambda : 0)
for idx, feature in enumerate(tfidf_vectorizer.get_feature_names()):
  word2id[feature] = idx

for idx, sent in enumerate(all_sentences_horror):
  #print(f'---공포 소설 {idx} 번---')
  results_horror = [(token, tfidf_matrix[idx, word2id[token]]) for token in sent.split()]
  results_horror.sort(key = lambda element : element[1], reverse = True)
  #print(results_horror)
  #print("\n")

In [34]:
#판타지
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(all_sentences_fantasy)

word2id = defaultdict(lambda : 0)
for idx, feature in enumerate(tfidf_vectorizer.get_feature_names()):
  word2id[feature] = idx

for idx, sent in enumerate(all_sentences_fantasy):
  #print(f'--- 판타지 소설 {idx} 번---')
  results_fantasy = [(token, tfidf_matrix[idx, word2id[token]]) for token in sent.split()]
  results_fantasy.sort(key = lambda element : element[1], reverse = True)
  #print(results_fantasy)
  #print("\n")

In [35]:
#무협
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(all_sentences_martial)

word2id = defaultdict(lambda : 0)
for idx, feature in enumerate(tfidf_vectorizer.get_feature_names()):
  word2id[feature] = idx
  
for idx, sent in enumerate(all_sentences_martial):
  #print(f'--- 무협 소설 {idx} 번---')
  results_martial = [(token, tfidf_matrix[idx, word2id[token]]) for token in sent.split()]
  results_martial.sort(key = lambda element : element[1], reverse = True)
  #print(results_martial)
  #print("\n")

In [36]:
#sf
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(all_sentences_sf)

word2id = defaultdict(lambda : 0)
for idx, feature in enumerate(tfidf_vectorizer.get_feature_names()):
  word2id[feature] = idx

for idx, sent in enumerate(all_sentences_sf):
  #print(f'--- sf 소설 {idx} 번---')
  results_sf = [(token, tfidf_matrix[idx, word2id[token]]) for token in sent.split()]
  results_sf.sort(key = lambda element : element[1], reverse = True)
  #print(results_sf)
  #print("\n")

In [37]:
#역사
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(all_sentences_history)

word2id = defaultdict(lambda : 0)
for idx, feature in enumerate(tfidf_vectorizer.get_feature_names()):
  word2id[feature] = idx
  
for idx, sent in enumerate(all_sentences_history):
  #print(f'--- sf 소설 {idx} 번---')
  results_history = [(token, tfidf_matrix[idx, word2id[token]]) for token in sent.split()]
  results_history.sort(key = lambda element : element[1], reverse = True)
  #print(results_history)
  #print("\n")

In [38]:
#연애
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(all_sentences_romance)

word2id = defaultdict(lambda : 0)
for idx, feature in enumerate(tfidf_vectorizer.get_feature_names()):
  word2id[feature] = idx
for idx, sent in enumerate(all_sentences_romance):
  #print(f'--- 연애 소설 {idx} 번---')
  results_romance = [(token, tfidf_matrix[idx, word2id[token]]) for token in sent.split()]
  results_romance.sort(key = lambda element : element[1], reverse = True)
  #print(results_romance)
  #print("\n")

In [39]:
print(results_romance)
print(results_fantasy)
print(results_history)
print(results_horror)
print(results_martial)
print(results_sf)
print(results_mystery)

[]
[('태곳적', 0.41762871029104454), ('불로불사', 0.41762871029104454), ('고통', 0.3216154876463946), ('행복', 0.29615096110402), ('만큼', 0.2855604804698291), ('누구', 0.24683738179823825), ('그것', 0.24129192930211454), ('남자', 0.22841464613174067), ('상상', 0.22404094675088906), ('시간', 0.2151903065671379), ('존재', 0.2064191634715358), ('자신', 0.18861900678059348), ('사람', 0.18020784007012874), ('힘', 0.0), ('그', 0.0)]
[('천손', 0.24051687800703264), ('제천', 0.24051687800703264), ('상고사', 0.24051687800703264), ('주민', 0.24051687800703264), ('신시', 0.24051687800703264), ('마고성', 0.24051687800703264), ('신성', 0.23303495878332126), ('교환', 0.23303495878332126), ('물품', 0.23303495878332126), ('화합', 0.23303495878332126), ('종족', 0.23303495878332126), ('공간', 0.22692178565938517), ('장소', 0.22175317142897116), ('고도', 0.22175317142897116), ('교류', 0.20979399490805214), ('의식', 0.16778932445472153), ('하늘', 0.16088718261589338), ('문화', 0.15583014793380254), ('등장', 0.15203027980316383), ('기록', 0.12969967836426766), ('하나', 0.1296996

# 유사도 검증 

In [40]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.metrics.pairwise import linear_kernel

In [41]:
df_books_data

,id,title,author,pub,date,price,review,rating,story,img,genre
0,1,요리코를 위해,노리즈키 린타로,모모,2020.03,13500,43,9.2,스스로 악마이자 신이 되어 딸을 살해한 남자를 심판한 아버지.비극의 출발이자 마지막...,http://image.yes24.com/goods/89392792/L,추리/미스터리
1,2,그녀는 다 계획이 있다,히가시노 게이고,하빌리스,2021.02,14220,13,9.4,하나야 보석점 살인사건을 둘러싼 과거와 미래의 사건경쾌하고 리듬감 있는 터치의 히가...,http://image.yes24.com/goods/97561319/L,추리/미스터리
2,3,블랙 쇼맨과 이름 없는 마을의 살인,히가시노 게이고,알에이치코리아(RHK),2020.11,16200,54,9.1,"아버지가 살해당했다, 그것도 결혼 소식을 알린 직후에. 행복하다고 느끼는 순간, 비...",http://image.yes24.com/goods/95712763/L,추리/미스터리
3,4,돌이킬 수 없는 약속,야쿠마루 가쿠,북플라자,2017.02,13500,217,8.2,"제51회 에드가와란포상 수상 작가 야쿠마루 가쿠의 미스터리 추리 소설. ""내 딸을 ...",http://image.yes24.com/goods/35552641/L,추리/미스터리
4,5,루팡의 딸,요코제키 다이,북플라자,2019.09,13500,31,8.5,도둑 집안의 대부가 변사체로 발견되었다!도둑 집안의 대부 이와오에게는 하나코라는 손...,http://image.yes24.com/goods/78860364/L,추리/미스터리
...,...,...,...,...,...,...,...,...,...,...,...
79741,79742,나는 그놈의 전부였다 1,,청어람,2003.05,7200,3,5.7,,http://image.yes24.com/goods/362884/L,로맨스
79761,79762,맥그리거의 운명,노라 노버츠,신영미디어,2003.06,7650,3,7.3,돈독한 우애를 자랑하는 맥그리거 남매들! 케인의 여동생 세레나와 다이애너의 오빠가 ...,http://image.yes24.com/goods/370757/L,로맨스
79762,79763,악마의 키스,,신영미디어,2003.06,8100,2,8.5,"""스테파니 로렌스가 전하는 신스터(Cynster) 가문 이야기! ?바 신스터?일당의...",http://image.yes24.com/goods/373141/L,로맨스
79769,79770,하루만 사랑해 1,,파피루스(디앤씨미디어),2003.06,8100,1,7.0,어느 날 서원은 횡단보도에서 교통사고를 당한다. 상대는 인천상고 짱인 노란머 민하루...,http://image.yes24.com/goods/375482/L,로맨스


In [42]:
df_books

,index,id,title,author,pub,date,price,review,rating,story,img,genre
0,0,1,요리코를 위해,노리즈키 린타로,모모,2020.03,13500,43,9.2,스스로 악마이자 신이 되어 딸을 살해한 남자를 심판한 아버지.비극의 출발이자 마지막...,http://image.yes24.com/goods/89392792/L,추리/미스터리
1,1,2,그녀는 다 계획이 있다,히가시노 게이고,하빌리스,2021.02,14220,13,9.4,하나야 보석점 살인사건을 둘러싼 과거와 미래의 사건경쾌하고 리듬감 있는 터치의 히가...,http://image.yes24.com/goods/97561319/L,추리/미스터리
2,2,3,블랙 쇼맨과 이름 없는 마을의 살인,히가시노 게이고,알에이치코리아(RHK),2020.11,16200,54,9.1,"아버지가 살해당했다, 그것도 결혼 소식을 알린 직후에. 행복하다고 느끼는 순간, 비...",http://image.yes24.com/goods/95712763/L,추리/미스터리
3,3,4,돌이킬 수 없는 약속,야쿠마루 가쿠,북플라자,2017.02,13500,217,8.2,"제51회 에드가와란포상 수상 작가 야쿠마루 가쿠의 미스터리 추리 소설. ""내 딸을 ...",http://image.yes24.com/goods/35552641/L,추리/미스터리
4,4,5,루팡의 딸,요코제키 다이,북플라자,2019.09,13500,31,8.5,도둑 집안의 대부가 변사체로 발견되었다!도둑 집안의 대부 이와오에게는 하나코라는 손...,http://image.yes24.com/goods/78860364/L,추리/미스터리
...,...,...,...,...,...,...,...,...,...,...,...,...
18813,79741,79742,나는 그놈의 전부였다 1,,청어람,2003.05,7200,3,5.7,,http://image.yes24.com/goods/362884/L,로맨스
18814,79761,79762,맥그리거의 운명,노라 노버츠,신영미디어,2003.06,7650,3,7.3,돈독한 우애를 자랑하는 맥그리거 남매들! 케인의 여동생 세레나와 다이애너의 오빠가 ...,http://image.yes24.com/goods/370757/L,로맨스
18815,79762,79763,악마의 키스,,신영미디어,2003.06,8100,2,8.5,"""스테파니 로렌스가 전하는 신스터(Cynster) 가문 이야기! ?바 신스터?일당의...",http://image.yes24.com/goods/373141/L,로맨스
18816,79769,79770,하루만 사랑해 1,,파피루스(디앤씨미디어),2003.06,8100,1,7.0,어느 날 서원은 횡단보도에서 교통사고를 당한다. 상대는 인천상고 짱인 노란머 민하루...,http://image.yes24.com/goods/375482/L,로맨스


In [43]:
df_books_data['story'].isnull().sum()

0

In [44]:
df_books['story'] = df_books['story'].fillna('')

In [45]:
tfidf = TfidfVectorizer()
# overview에 대해서 tf-idf 수행
tfidf_matrix = tfidf.fit_transform(df_books_data['story'])
print(tfidf_matrix.shape)

(18818, 130823)


In [46]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [47]:
indices = pd.Series(df_books.index, index=df_books['title']).drop_duplicates()
print(indices.head())

title
요리코를 위해                0
그녀는 다 계획이 있다           1
블랙 쇼맨과 이름 없는 마을의 살인    2
돌이킬 수 없는 약속            3
루팡의 딸                  4
dtype: int64


In [41]:
idx = indices['요리코를 위해']
print(idx)

0


In [47]:
idx2 = indices['그녀는 다 계획이 있다']
print(idx2)

1


In [ ]:
idx3 = indices['루팡의 딸']
print(idx3)

In [43]:
indices

title
요리코를 위해                    0
그녀는 다 계획이 있다               1
블랙 쇼맨과 이름 없는 마을의 살인        2
돌이킬 수 없는 약속                3
루팡의 딸                      4
                       ...  
범상치 않은 관계 1            11151
보통의 세계                 11152
미주부동산 2                11153
미주부동산 1                11154
나쁜 결혼                  11155
Length: 11156, dtype: int64

In [63]:
def get_recommendations(title, cosine_sim=cosine_sim):
    # for i in range(title):
        # 선택된 책의 장르로부터 해당되는 인덱스를 받아옵니다. 이제 선택한 장르를 가지고 연산할 수 있습니다.
        idx = indices[title]
        
        # 모든 에 대해서 해당 영화와의 유사도를 구합니다.
        sim_scores = list(enumerate(cosine_sim[idx]))

        # 유사도에 따라 영화들을 정렬합니다.
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

        # 가장 유사한 10개의 영화를 받아옵니다.
        sim_scores = sim_scores[1:11]

        # 가장 유사한 10개의 영화의 인덱스를 받아옵니다.
        movie_indices = [i[0] for i in sim_scores]

        # 가장 유사한 10개의 영화의 제목을 리턴합니다.
        return df_books['title'].iloc[movie_indices]

In [64]:
get_recommendations('요리코를 위해')

4275        이노엘리 2
2772        블랙 러시안
3831    괴물 이야기 (하)
4612    괴물 이야기 (상)
1715    소풍 버스 납치사건
8705        너는 누구니
2771        복수의 미학
1205         1의 비극
9681      괴물이 산다 3
9682      괴물이 산다 2
Name: title, dtype: object

In [71]:
def get_recommendations2(title, cosine_sim=cosine_sim):
    # for i in range(title):
        # 선택된 책의 장르로부터 해당되는 인덱스를 받아옵니다. 이제 선택한 장르를 가지고 연산할 수 있습니다.
        idx = indices[title]
        
        # 모든 에 대해서 해당 영화와의 유사도를 구합니다.
        sim_scores = list(enumerate(cosine_sim[idx]))

        # 유사도에 따라 영화들을 정렬합니다.
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

        # 가장 유사한 10개의 영화를 받아옵니다.
        sim_scores = sim_scores[1:2]

        # 가장 유사한 10개의 영화의 인덱스를 받아옵니다.
        movie_indices = [i[0] for i in sim_scores]

        # 가장 유사한 10개의 영화의 제목을 리턴합니다.
        return df_books['title'].iloc[movie_indices]

In [72]:
get_recommendations2('루팡의 딸')

2905    어둠의 도둑
Name: title, dtype: object

In [67]:
def get_recommendations3(title, cosine_sim=cosine_sim):
    # for i in range(title):
        # 선택된 책의 장르로부터 해당되는 인덱스를 받아옵니다. 이제 선택한 장르를 가지고 연산할 수 있습니다.
        idx = indices[title]
        
        # 모든 에 대해서 해당 영화와의 유사도를 구합니다.
        sim_scores = list(enumerate(cosine_sim[idx]))

        # 유사도에 따라 영화들을 정렬합니다.
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

        # 가장 유사한 10개의 영화를 받아옵니다.
        sim_scores = sim_scores[1:11]

        # 가장 유사한 10개의 영화의 인덱스를 받아옵니다.
        movie_indices = [i[0] for i in sim_scores]

        # 가장 유사한 10개의 영화의 제목을 리턴합니다.
        return df_books['title'].iloc[movie_indices]

In [68]:
get_recommendations3('그녀는 다 계획이 있다')

1809             악마, 할로윈 파티에서 죽다
1296                    보라색 위크엔드
2709                    청계산장의 재판
10107                 파티시엘 강나예 2
10108                 파티시엘 강나예 1
414                      46번째 밀실
19                    하쿠바산장 살인사건
2551             다잉 아이 Dying Eye
8640                    인소의 법칙 6
9786     인소의 법칙 6 오디오 드라마 CD 한정판
Name: title, dtype: object